In [1]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
real_y= pd.read_csv('RealSeoulPredictions.csv')

print(train_df.shape)
print(test_df.shape)


(5760, 15)
(3000, 14)


In [3]:
y_train=train_df['y']
x_train=train_df.drop(['ID','y'],axis=1)
x_valid=test_df.drop('ID',axis=1)
#x_test.isna().sum()
x_train

,Date,Hour,Temperature(�C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(�C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
0,01/12/2017,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
1,01/12/2017,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
2,01/12/2017,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes
3,01/12/2017,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
4,01/12/2017,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5755,20/11/2018,19,8.7,34,2.2,1857,-6.3,0.0,0.0,0.0,Autumn,No Holiday,Yes
5756,20/11/2018,20,8.5,32,2.6,1789,-7.2,0.0,0.0,0.0,Autumn,No Holiday,Yes
5757,20/11/2018,21,8.8,37,3.8,1750,-5.1,0.0,0.0,0.0,Autumn,No Holiday,Yes
5758,20/11/2018,22,8.9,40,3.4,1810,-3.9,0.0,0.0,0.0,Autumn,No Holiday,Yes


In [4]:
#x= (train_df.columns)
#for i in x:
#    plt.figure()
#    plt.subplot(121)
#    plt.scatter(y_train[:100],train_df[i][:100])
#    plt.title(train_df[i].name)
#    plt.show()

In [5]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in x_train.columns if x_train[cname].nunique() < 10 and 
                        x_train[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in x_train.columns if x_train[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numerical_cols

# Get list of categorical variables
s = (x_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(x_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(x_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = x_train.index
OH_cols_valid.index = x_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = x_train.drop(object_cols, axis=1)
num_X_valid = x_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

#print("MAE from Approach 3 (One-Hot Encoding):") 
#print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))


Categorical variables:
['Date', 'Seasons', 'Holiday', 'Functioning Day']


In [6]:
#columns converting
import datetime as dt

x_train['Functioning Day']=x_train['Functioning Day'].map(dict(Yes=1, No=0))

Holidays_map = {'Holiday': 1, 'No Holiday': 0}
x_train['Holiday']=x_train['Holiday'].map(Holidays_map)

seasons_map = {'Summer': 0, 'Autumn': 1,'Winter':2,'Spring':3}
x_train['Seasons']=x_train['Seasons'].map(seasons_map)

x_train['Date'] = pd.to_datetime(x_train['Date'])
#pd.to_numeric(test_df['Date'])
x_train['Date']=x_train['Date'].map(dt.datetime.toordinal)

#x_train= (x_train-x_train.mean()) / x_train.std()
x_train

,Date,Hour,Temperature(�C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(�C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
0,736341,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,2,0,1
1,736341,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,2,0,1
2,736341,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,2,0,1
3,736341,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,2,0,1
4,736341,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5755,737018,19,8.7,34,2.2,1857,-6.3,0.0,0.0,0.0,1,0,1
5756,737018,20,8.5,32,2.6,1789,-7.2,0.0,0.0,0.0,1,0,1
5757,737018,21,8.8,37,3.8,1750,-5.1,0.0,0.0,0.0,1,0,1
5758,737018,22,8.9,40,3.4,1810,-3.9,0.0,0.0,0.0,1,0,1


In [7]:
x_valid['Functioning Day']=x_valid['Functioning Day'].map(dict(Yes=1, No=0))

Holidays_map = {'Holiday': 1, 'No Holiday': 0}
x_valid['Holiday']=x_valid['Holiday'].map(Holidays_map)

seasons_map = {'Summer': 0, 'Autumn': 1,'Winter':2,'Spring':3}
x_valid['Seasons']=x_valid['Seasons'].map(seasons_map)

x_valid['Date'] = pd.to_datetime(x_valid['Date'])
x_valid['Date']=x_valid['Date'].map(dt.datetime.toordinal)

#x_valid= (x_valid-x_valid.mean()) / x_valid.std()
x_valid

,Date,Hour,Temperature(�C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(�C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
0,736684,0,-1.3,90,2.0,259,-2.7,0.0,0.0,2.2,2,0,1
1,736684,1,-1.8,90,1.6,263,-3.2,0.0,0.0,2.2,2,0,1
2,736684,2,-2.2,88,1.4,348,-3.9,0.0,0.0,2.2,2,0,1
3,736684,3,-2.9,89,1.3,288,-4.4,0.0,0.0,2.2,2,0,1
4,736684,4,-3.6,91,1.2,241,-4.8,0.0,0.0,2.2,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,737028,19,4.2,34,2.6,1894,-10.3,0.0,0.0,0.0,1,0,1
2996,737028,20,3.4,37,2.3,2000,-9.9,0.0,0.0,0.0,1,0,1
2997,737028,21,2.6,39,0.3,1968,-9.9,0.0,0.0,0.0,1,0,1
2998,737028,22,2.1,41,1.0,1859,-9.8,0.0,0.0,0.0,1,0,1


In [8]:
from sklearn.ensemble import GradientBoostingRegressor
#reg = GradientBoostingRegressor(random_state=0)
#reg.fit(x_train, y_train)
#preds=reg.predict(x_valid)
#test_df['y']=preds
#OH_X_train , OH_X_valid

from sklearn.preprocessing import PolynomialFeatures
#from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
#model = Pipeline([('poly', PolynomialFeatures(degree=3)) , ('linear', RandomForestRegressor(n_estimators=10, random_state=0))])
#model = model.fit(x_train, y_train)
#preds=model.predict(x_valid)
#test_df['y']=preds

from xgboost import XGBRegressor

my_model =Pipeline([('poly', PolynomialFeatures(degree=4)) , ( 'linear',XGBRegressor(n_estimators=300, learning_rate=0.02, n_jobs=4))])

my_model.fit(x_train, y_train)
preds=my_model.predict(x_valid)
test_df['y']=preds

# Function for comparing different approaches
def score_dataset(x_train, x_valid, y_train):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(x_train, y_train)
    preds = model.predict(x_valid)
    return preds
#test_df['y']=score_dataset(OH_X_train , OH_X_valid, y_train)


In [9]:
final_df = test_df[['ID', 'y']].copy()
final_df.to_csv('output.csv', index=False)
print('done')

done


In [10]:
#final_df['y'] , real_y['y']
def rmsle(Y, y_pred):
    sum=0.0
    for x in range(len(y_pred)):
        if y_pred[x]<0 or Y[x]<0: #check for negative values
            continue
        p = np.log(y_pred[x]+1)
        r = np.log(Y[x]+1)
        sum = sum + (p - r)**2
    return (sum/len(y_pred))**0.5
rmsle( real_y['y'], final_df['y'] )

0.6104718188533116